In [ ]:
import pandas as pd
import numpy as np
from rest_framework.views import APIView
from rest_framework.authentication import get_authorization_header
from rest_framework.response import Response
import time
import traceback
import matplotlib.pyplot as plt
plt.style.use('Solarize_Light2')
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
class timeseries_resampling(APIView):
    def post(self,request):
        auth_keyword, token = get_authorization_header(request).split()
        token = token.decode("utf-8")
        check = headercheck(token)  #upload,preview and stats
        if check == 'ok':
            pass
        else:
            return Response({"status": "validation", "error": "user has been killed"})
        data = request.data
        analysis = data['analysis_name']
        filename = data['filename']
        version = data['version']
        username = data['username']
        date=data['date']
        resamplemethod= data['resamplemethod']
        resampletype= data['resampletype']
        frequencytype= data['frequencytype']
        imputation =data['imputation']
        try:
            import numpy as np
            file = match_version(version, analysis, filename)
            data = read_file(file)  # spark data frame
            if str(type(data)) == "<class 'str'>":
                data = read_file(filename)
            df1 = data.toPandas()
            df1[date] = pd.to_datetime(df1[date])
            df1 = df1.sort_values(by=date)
            weekday = df1[date].dt.dayofweek
            weekday1 = weekday[len(weekday) - 1]
            def weekdays(weekday1):
                if weekday1 == 0:
                    day = 'W-MON'
                elif weekday1 == 1:
                    day = 'W-TUE'
                elif weekday1 == 2:
                    day = 'W-WED'
                elif weekday1 == 3:
                    day = 'W-THU'
                elif weekday1 == 4:
                    day = 'W-FRI'
                elif weekday1 == 5:
                    day = 'W-SAT'
                elif weekday1 == 6:
                    day = 'W-SUN'
                return day
            df1.set_index(date, inplace=True)
            if resamplemethod == 'no':
                pass
            elif resamplemethod == 'upsample':
                if resampletype == 'no':
                    pass
                elif resampletype == 'linear':
                    second = df1.resample('S').interpolate(method='linear')
                    minute = df1.resample('T').interpolate(method='linear')
                    hourly = df1.resample('H').interpolate(method='linear')  # two columns( one date column and one target column) as the base data frame.
                    daily = df1.resample('D').interpolate(method='linear')
                    weekly = df1.resample(weekdays(weekday1)).interpolate(method='linear')
                    monthly = df1.resample('M').interpolate(method='linear')
                    quarterly = df1.resample('Q').interpolate(method='linear')
                    yearly = df1.resample('Y').interpolate(method='linear')

                elif resampletype == 'spline':
                    seconds = df1.resample('S').interpolate(method='spline', order=2)
                    minutes = df1.resample('T').interpolate(method='spline', order=2)
                    hourly = df1.resample('H').interpolate(method='spline',order=2)  # two columns( one date column and one target column) as the base data frame.
                    daily = df1.resample('D').interpolate(method='spline', order=2)
                    weekly = df1.resample(weekdays(weekday1)).interpolate(method='spline', order=2)
                    monthly = df1.resample('M').interpolate(method='spline', order=2)
                    quarterly = df1.resample('Q').interpolate(method='spline', order=2)
                    yearly = df1.resample('Y').interpolate(method='spline', order=2)

                elif resampletype == 'polynomial':
                    second = df1.resample('S').interpolate(method='polynomial', order=2)
                    minute = df1.resample('T').interpolate(method='polynomial', order=2)
                    hourly = df1.resample('H').interpolate(method='polynomial',order=2)  # two columns( one date column and one target column) as the base data frame.
                    daily = df1.resample('D').interpolate(method='polynomial', order=2)
                    weekly = df1.resample(weekdays(weekday1)).interpolate(method='polynomial', order=2)
                    monthly = df1.resample('M').interpolate(method='polynomial', order=2)
                    quarterly = df1.resample('Q').interpolate(method='polynomial', order=2)
                    yearly = df1.resample('Y').interpolate(method='polynomial', order=2)

                elif resampletype == 'cubic':
                    second = df1.resample('S').interpolate(method='cubic')
                    minute = df1.resample('T').interpolate(method='cubic')
                    hourly = df1.resample('H').interpolate(method='cubic')  # two columns( one date column and one target column) as the base data frame.
                    daily = df1.resample('D').interpolate(method='cubic')
                    weekly = df1.resample(weekdays(weekday1)).interpolate(method='cubic')
                    monthly = df1.resample('M').interpolate(method='cubic')
                    quarterly = df1.resample('Q').interpolate(method='cubic')
                    yearly = df1.resample('Y').interpolate(method='cubic')

                elif resampletype == 'quadratic':
                    second = df1.resample('S').interpolate(method='quadratic')
                    minute = df1.resample('T').interpolate(method='quadratic')
                    hourly = df1.resample('H').interpolate(method='quadratic')  # two columns( one date column and one target column) as the base data frame.
                    daily = df1.resample('D').interpolate(method='quadratic')
                    weekly = df1.resample(weekdays(weekday1)).interpolate(method='quadratic')
                    monthly = df1.resample('M').interpolate(method='quadratic')
                    quarterly = df1.resample('Q').interpolate(method='quadratic')
                    yearly = df1.resample('Y').interpolate(method='quadratic')


            elif resamplemethod == 'downsample':
                if resampletype == 'no':
                    pass
                elif resampletype == 'mean':
                    second = df1.resample('S').mean()
                    minute = df1.resample('T').mean()
                    hourly = df1.resample('H').mean()
                    daily = df1.resample('D').mean()
                    weekly = df1.resample(weekdays(weekday1)).mean()
                    monthly = df1.resample('M').mean()
                    quarterly = df1.resample('Q').mean()
                    yearly = df1.resample('Y').mean()

                elif resampletype == 'sum':
                    second = df1.resample('S').sum()
                    minute = df1.resample('T').sum()
                    hourly = df1.resample('H').sum()
                    daily = df1.resample('D').sum()
                    weekly = df1.resample(weekdays(weekday1)).sum()
                    monthly = df1.resample('M').sum()
                    quarterly = df1.resample('Q').sum()
                    yearly = df1.resample('Y').sum()

                elif resampletype == 'min':
                    second = df1.resample('S').min()
                    minute = df1.resample('T').min()
                    hourly = df1.resample('H').min()
                    daily = df1.resample('D').min()
                    weekly = df1.resample(weekdays(weekday1)).min()
                    monthly = df1.resample('M').min()
                    quarterly = df1.resample('Q').min()
                    yearly = df1.resample('Y').min()

                elif resampletype == 'max':
                    second = df1.resample('S').max()
                    minute = df1.resample('T').max()
                    hourly = df1.resample('H').max()
                    daily = df1.resample('D').max()
                    weekly = df1.resample(weekdays(weekday1)).max()
                    monthly = df1.resample('M').max()
                    quarterly = df1.resample('Q').max()
                    yearly = df1.resample('Y').max()

                elif resampletype == 'count':
                    second = df1.resample('S').count()
                    minute = df1.resample('T').count()
                    hourly = df1.resample('H').count()
                    daily = df1.resample('D').count( )
                    weekly = df1.resample(weekdays(weekday1)).count()
                    monthly = df1.resample('M').count()
                    quarterly = df1.resample('Q').count()
                    yearly = df1.resample('Y').count()

            if frequencytype == 'second':
                if resampletype == 'no':
                    pass
                else:
                    df2 = second
            elif frequencytype == 'minute':
                if resampletype == 'no':
                    pass
                else:
                    df2 = minute
            elif frequencytype == 'hourly':
                if resampletype == 'no':
                    pass
                else:
                    df2 = hourly
            elif frequencytype == 'daily':
                if resampletype == 'no':
                    pass
                else:
                    df2 = daily
            elif frequencytype == 'weekly':
                if resampletype == 'no':
                    pass
                else:
                    df2 = weekly
            elif frequencytype == 'monthly':
                if resampletype == 'no':
                    pass
                else:
                    df2 = monthly
            elif frequencytype == 'quarterly':
                if resampletype == 'no':
                    pass
                else:
                    df2 = quarterly
            elif frequencytype == 'yearly':
                if resampletype == 'no':
                    pass
                else:
                    df2 = yearly

            if imputation == 'no':
                df3 = df1
                del df1
            elif imputation == 'mean':
                df3 = df2.fillna(df2.mean())
                del df2
            elif imputation == 'median':
                df3 = df2.fillna(df2.median())
                del df2

            return df3

        except Exception as e:
            traceback.print_exc()
            return Response({"status": "failure","error":"Please refer the logs in Logs->Container Log"})
